## Implementing a data source: retrieving data from a public API, mapping it to a small data model, and storing it in a Mongo database

In [1]:
import requests
import json
import os
import sys
import pandas as pd


In [4]:
schema = {
 "$schema": "http://json-schema.org/draft-07/schema#",
 "title": "Clinical Trial Metadata",
 "type": "object",
 "properties": {
 "trialId": {
 "type": "string",
 "description": "A unique identifier for the clinical trial."
 },
 "title": {
 "type": "string",
 "description": "The official title of the clinical trial."
 },
 "startDate": {
 "type": "string",
 "format": "date",
 "description": "The start date of the clinical trial."
 },
 "endDate": {
 "type": "string",
 "format": "date",
 "description": "The end date of the clinical trial, if applicable."
 },
 "phase": {
 "type": "string",
 "enum": ["Phase 1", "Phase 2", "Phase 3", "Phase 4", "Other"],
 "description": "The phase of the clinical trial."
 },
 "principalInvestigator": {
 "type": "object",
 "properties": {
 "name": {"type": "string",
 "description": "The name of the principal investigator."
 },
 "affiliation": {
 "type": "string",
 "description": "The affiliation of the principal investigator."
 }
 },
 "required": ["name"]
 },
 "locations": {
 "type": "array",
 "items": {
 "type": "object",
 "properties": {
 "facility": {
 "type": "string",
 "description": "Name of the facility where the trial is conducted."
 },
 "city": {
 "type": "string",
 "description": "The city where the facility is located."
 },
 "country": {
 "type": "string",
 "description": "The country where the facility is located."
 }
 }
 }
 },
 "eligibilityCriteria": {
 "type": "string",
 "description": "A description of the eligibility criteria for the trial."
 }
 },
 "required": ["trialId", "title", "phase"]
}


In [2]:
import pprint 

pp = pprint.PrettyPrinter(sort_dicts=True)
# pp.pprint(schema)

### Using the clinicaltrials.gov API, retrieve all studies that have been last updated between 20 and 21 October 2024. (A “last update” date is available per study.)

In [10]:
import requests

# Base URL for ClinicalTrials.gov API
base_url = "https://clinicaltrials.gov/api/v2/studies"


# Parameters for the query
params = {
    "format": "json",  # Requesting JSON format
    "query.term": "AREA[LastUpdatePostDate]RANGE[2024-10-20,2024-10-21]",  # Essie expression
}



# Initialize the next page token
next_page_token = None

# Initialize an empty list to store the data
data_list = []

while True:
    if next_page_token:
        # Add the nextPageToken to the parameters for subsequent requests
        params["pageToken"] = next_page_token
        
    # Sending the request
    response = requests.get(base_url, params=params)
    
    # Handling the response
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        studies = data.get('studies', [])  # Extract the list of studies

        current_path = os.getcwd()
        data_path = os.path.join(current_path, "rawdata")
        # Append studies to the data list
        data_list.extend(studies)
        
        page_filename = f"studies_page_{next_page_token}.json" if next_page_token else "studies_page_1.json"
        # dump to a json file
        
        with open(os.path.join(data_path, page_filename), "w") as file:
            json.dump(data, file, indent=4)

    #   Check for the nextPageToken
        next_page_token = data.get("nextPageToken")
        print(next_page_token)
        
        # Update the parameters with the nextPageToken
        params["pageToken"] = next_page_token
        
        if not next_page_token:
            print("No more pages to fetch.")
            break
            
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

# After all pages are fetched, save all data into a single file
with open(os.path.join(data_path, "all_studies.json"), "w") as file:
    json.dump(data_list, file, indent=4)

print(f"Fetched a total of {len(data_list)} studies.")

#     break
#     # Loop through each study and extract specific information
#     for study in studies:
#         # Safely access nested keys
#         nctId = study['protocolSection']['identificationModule'].get('nctId', 'Unknown')
#         overallStatus = study['protocolSection']['statusModule'].get('overallStatus', 'Unknown')
#         startDate = study['protocolSection']['statusModule'].get('startDateStruct', {}).get('date', 'Unknown Date')
#         conditions = ', '.join(study['protocolSection']['conditionsModule'].get('conditions', ['No conditions listed']))
#         acronym = study['protocolSection']['identificationModule'].get('acronym', 'Unknown')

#         # Extract interventions safely
#         interventions_list = study['protocolSection'].get('armsInterventionsModule', {}).get('interventions', [])
#         interventions = ', '.join([intervention.get('name', 'No intervention name listed') for intervention in interventions_list]) if interventions_list else "No interventions listed"

#         # Extract locations safely
#         locations_list = study['protocolSection'].get('contactsLocationsModule', {}).get('locations', [])
#         locations = ', '.join([f"{location.get('city', 'No City')} - {location.get('country', 'No Country')}" for location in locations_list]) if locations_list else "No locations listed"

#         # Extract dates and phases
#         primaryCompletionDate = study['protocolSection']['statusModule'].get('primaryCompletionDateStruct', {}).get('date', 'Unknown Date')
#         studyFirstPostDate = study['protocolSection']['statusModule'].get('studyFirstPostDateStruct', {}).get('date', 'Unknown Date')
#         lastUpdatePostDate = study['protocolSection']['statusModule'].get('lastUpdatePostDateStruct', {}).get('date', 'Unknown Date')
#         studyType = study['protocolSection']['designModule'].get('studyType', 'Unknown')
#         phases = ', '.join(study['protocolSection']['designModule'].get('phases', ['Not Available']))

#  # Append the data to the list as a dictionary
#         data_list.append({
#                 "NCT ID": nctId,
#                 "Acronym": acronym,
#                 "Overall Status": overallStatus,
#                 "Start Date": startDate,
#                 "Conditions": conditions,
#                 "Interventions": interventions,
#                 "Locations": locations,
#                 "Primary Completion Date": primaryCompletionDate,
#                 "Study First Post Date": studyFirstPostDate,
#                 "Last Update Post Date": lastUpdatePostDate,
#                 "Study Type": studyType,
#                 "Phases": phases
#             })
#         print(lastUpdatePostDate)
#         # Check for nextPageToken and update the params or break the loop
#         nextPageToken = data.get('nextPageToken')
#         if nextPageToken:
#             params['pageToken'] = nextPageToken  # Set the pageToken for the next request
#         else:
#             break  # Exit the loop if no nextPageToken is present
# else:
#     print("Failed to fetch data. Status code:", response.status_code)


# # Create a DataFrame from the list of dictionaries
# df = pd.DataFrame(data_list)

# # Print the DataFrame
# print(df)

NF0g5JuGl_gq
NF0g5JKBlvcqwQ
NF0g5JGDlvgowg
NF0g5JCHlfEsxw
NF0g5JCClvguwg
NF0g5JCOkfUrww
NF0g5JeBkfIvwQ
NF0g5JaGl_csww
NF0g5JaBlPIgxQ
NF0g5JWDm_UowQ
NF0g5JWOlPIvxw
NF0g5JSOlfIryA
NF0g5JuDm_Qtww
NF0g5JqGkfEgxw
NF0g5JKHkvIrwQI
NF0g5JKHmvYtxg4
NF0g5JKGkPUtwQM
NF0g5JKGlfEpxQU
NF0g5JKFkPYsxQI
NF0g5JKEkvAuyAY
NF0g5JKEl_AvyQA
NF0g5JKDkvgtwgI
NF0g5JKDl_AswQA
NF0g5JKCkvYowwM
NF0g5JKClPcgxwM
NF0g5JKBlPAvxw4
NF0g5JKAkvEgyAY
NF0g5JKAlvAhxA4
NF0g5JKPkvcowAI
NF0g5JKPlfIsyA8
NF0g5JKOkfUvxg8
NF0g5JKOm_UuxwM
NF0g5JGHmvQvyQE
NF0g5JGGlfUgxAI
NF0g5JGFlvMtyQA
NF0g5JGEkvcpwgc
NF0g5JGElPAoyQ8
NF0g5JGDkvYvwwY
NF0g5JGCkvQuyQA
NF0g5JGCl_ggwQE
NF0g5JGBlvEpxgU
NF0g5JGBm_UrxQI
NF0g5JGAlfcuwQE
NF0g5JGPlvAuwQU
NF0g5JGPm_Uvxwc
NF0g5JGOl_EpxgI
NF0g5JCHkPMpww4
NF0g5JCHm_gswAE
NF0g5JCGl_Ihwwc
NF0g5JCFkPMswQc
NF0g5JCFm_crxQM
NF0g5JCElvguwwc
NF0g5JCDkPAhwwI
NF0g5JCDl_AgwQ4
NF0g5JCCkfMuxQQ
NF0g5JCClfkpxQc
NF0g5JCBlvUpxg4
None
No more pages to fetch.
Fetched a total of 572 studies.


# Read the content of the 572 studies from a combined all_studies.json file

In [16]:
# f.close()
with open(os.path.join(data_path, "all_studies.json")) as f:
    combined_studies = json.load(f)
#     print(combined_studies)
    


In [18]:
len(combined_studies)

572

In [21]:
schema = {
 "$schema": "http://json-schema.org/draft-07/schema#",
 "title": "Clinical Trial Metadata",
 "type": "object",
 "properties": {
 "trialId": {
 "type": "string",
 "description": "A unique identifier for the clinical trial."
 },
 "title": {
 "type": "string",
 "description": "The official title of the clinical trial."
 },
 "startDate": {
 "type": "string",
 "format": "date",
 "description": "The start date of the clinical trial."
 },
 "endDate": {
 "type": "string",
 "format": "date",
 "description": "The end date of the clinical trial, if applicable."
 },
 "phase": {
 "type": "string",
 "enum": ["Phase 1", "Phase 2", "Phase 3", "Phase 4", "Other"],
 "description": "The phase of the clinical trial."
 },
 "principalInvestigator": {
 "type": "object",
 "properties": {
 "name": {
      "type": "string",
 "description": "The name of the principal investigator."
 },
 "affiliation": {
 "type": "string",
 "description": "The affiliation of the principal investigator."
 }
 },
 "required": ["name"]
 },
 "locations": {
 "type": "array",
 "items": {
 "type": "object",
 "properties": {
 "facility": {
 "type": "string",
 "description": "Name of the facility where the trial is conducted."
 },
 "city": {
 "type": "string",
 "description": "The city where the facility is located."
 },
 "country": {
 "type": "string",
 "description": "The country where the facility is located."
 }
 }
 }
 },
 "eligibilityCriteria": {
 "type": "string",
 "description": "A description of the eligibility criteria for the trial."
 }
 },
 "required": ["trialId", "title", "phase"]
}


In [22]:
pp.pprint(schema)

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'properties': {'eligibilityCriteria': {'description': 'A description of the '
                                                       'eligibility criteria '
                                                       'for the trial.',
                                        'type': 'string'},
                'endDate': {'description': 'The end date of the clinical '
                                           'trial, if applicable.',
                            'format': 'date',
                            'type': 'string'},
                'locations': {'items': {'properties': {'city': {'description': 'The '
                                                                               'city '
                                                                               'where '
                                                                               'the '
                                                                        

In [23]:
example_schema = {
 "trialId": "NCT00560521",
 "title": "Effect of Continuous Positive Airway Pressure on Fluid Absorption Among Patients With Pleural Effusion Due to Tuberculosis",
 "startDate": "2005-03-01",
 "endDate": "2007-03-01",
 "phase": "Other",
 "principalInvestigator": {
 "name": "Juliana F Oliveira",
 "affiliation": "Universidade Federal do Rio de Janeiro"
 },
 "locations": [
 {
 "facility": "Federal University of Rio de Janeiro",
 "city": "Rio de Janeiro",
 "country": "Brazil"
 }
 ],
 "eligibilityCriteria": "Inclusion Criteria:\n\nConfirmed diagnosis of pleural tuberculosis.\nPatients 18 years of age and older.\n\nExclusion criteria:\n\nBe under previous treatment of respirat ory physiotherapy.\nIrregular use or abandonment of the anti-TB standard regimen.\nTo fail one or more physiotherapy section.\nTo fail one or more radiological evaluation."
}


## The study data structure was downloaded from ClinicalTrials.gov to search for the key names as given in the example schema for mapping.

In [85]:
example_schema.keys() # this needs to be extracted from the downloaded data

dict_keys(['trialId', 'title', 'startDate', 'endDate', 'phase', 'principalInvestigator', 'locations', 'eligibilityCriteria'])

In [179]:
import re
relevant_fields = {}        # relevant fields that look like keys in example schema 
for field in example_schema.keys():
#     print(field)
    if field == 'trialId':
        print(field)
        field_original = field
        field = 'id'
        result = [y for y in datastruct_protocol['Piece Name'] if re.search(f"{field}", str(y), re.IGNORECASE)]
        print(result)
#         print(datastruct_protocol[datastruct_protocol['Piece Name'].isin(result)][['Piece Name', 'Classic XPath']])
        relevant_fields.update({field_original: datastruct_protocol[datastruct_protocol['Piece Name'].isin(result)][['Piece Name', 'Classic XPath']]})
        
    elif field == 'endDate':
        print(field)
        field_original = field
        field = 'CompletionDate'
        result = [y for y in datastruct_protocol['Piece Name'] if re.search(f"{field}", str(y), re.IGNORECASE)]
        print(result)
        relevant_fields.update({field_original: datastruct_protocol[datastruct_protocol['Piece Name'].isin(result)][['Piece Name', 'Classic XPath']]})
        
    elif field == 'principalInvestigator':
        print(field)
        field_original = field
        field = 'Investigator'
        result = [y for y in datastruct_protocol['Piece Name'] if re.search(f"{field}", str(y), re.IGNORECASE)]
        print(result)
        relevant_fields.update({field_original: datastruct_protocol[datastruct_protocol['Piece Name'].isin(result)][['Piece Name', 'Classic XPath']]})
    
    
    else:
        print(field)
        result = [y for y in datastruct_protocol['Piece Name'] if re.search(f"{field}", str(y), re.IGNORECASE)]
        print(result)
        relevant_fields.update({field: datastruct_protocol[datastruct_protocol['Piece Name'].isin(result)][['Piece Name', 'Classic XPath']]})
    print()
    

trialId
['identificationModule', 'IdentificationModule', 'nctId', 'NCTId', 'nctIdAliases', 'NCTIdAlias', 'orgStudyIdInfo', 'OrgStudyIdInfo', 'id', 'OrgStudyId', 'OrgStudyIdType', 'OrgStudyIdLink', 'secondaryIdInfos', 'SecondaryIdInfo', 'id', 'SecondaryId', 'SecondaryIdType', 'SecondaryIdDomain', 'SecondaryIdLink', 'numSecondaryIds\xa0✗', 'NumSecondaryIds', 'nctId', 'ExpandedAccessNCTId', 'statusForNctId', 'ExpandedAccessStatusForNCTId', 'nPtrsToThisExpAccNctId', 'NPtrsToThisExpAccNCTId', 'individual', 'ExpAccTypeIndividual', 'pmid', 'ReferencePMID', 'pmid', 'RetractionPMID', 'id', 'AvailIPDId']

title
['briefTitle', 'BriefTitle', 'officialTitle', 'OfficialTitle', 'investigatorTitle', 'ResponsiblePartyInvestigatorTitle', 'oldNameTitle', 'ResponsiblePartyOldNameTitle']

startDate
['startDateStruct', 'StartDateStruct', 'StartDate', 'StartDateType']

endDate
['primaryCompletionDateStruct', 'PrimaryCompletionDateStruct', 'PrimaryCompletionDate', 'PrimaryCompletionDateType', 'completionDateS

In [510]:
## Manually written dict based on the keywords found from searching keys from example schema, required as the field names are different.
field_list = {'trialId':'nctId', 'title':'officialTitle', 'startDate':'startDateStruct', 'endDate':'completionDateStruct', 'phase':'Phase', 'principalInvestigator':['investigatorFullName',  'investigatorAffiliation'], 'locations':'locations\xa0⤷', 'eligibilityCriteria':'eligibilityCriteria'}


def xpath(a: str) -> str:
    '''function to modify XPath strings.'''
    if '/' in a:
        b = a.replace('/', '.')
        if 'Study' in b:
            return b.replace('.Study.', '')
    else:
        return a
    
    
# Doesn't require now
# Flattening the list 
# flattened_field_list = [item for sublist in field_list.values() for item in (sublist if isinstance(sublist, list) else [sublist])]

# print(flattened_field_list)
# filtered_df_2 = filtered_df[filtered_df["Piece Name"].isin(flattened_field_list)]


# filtered_df_2['modified_key'] = [[key for key, value in field_list.items() if (isinstance(value, list) and i in value) or (i == value)][0] for i in filtered_df_2['Piece Name']]
# print(filtered_df_2)

In [553]:
relevant_fields_df[relevant_fields_df['Piece Name'] == 'completionDateStruct']

,Piece Name,Classic XPath
74,completionDateStruct,protocolSection.statusModule.completionDateStruct


In [593]:

super_dict = {}
for n, x in enumerate(combined_studies):
#     print(n)
    mapped_data = {}
    flat_json_1 = flatten_json(combined_studies[n])
#     print(flat_json_1.keys())
    for item in field_list:
        i = field_list[item]
#         print(item, i, '__________________')
        
        if isinstance(i, list):
#             print(i)
            try:
                investigater = {}
                for f in i:
                    col_a = xpath(relevant_fields_df[relevant_fields_df['Piece Name'] == f]['Classic XPath'].values[0])
#                     print(col_a)
                    try:
                        investigater.update({f: flat_json_1[col_a]})
                    except:
                        pass
                mapped_data.update({item : investigater})
            except:
                pass
                        
        else:
            if item == 'locations':
                col_a = xpath(relevant_fields_df[relevant_fields_df['Piece Name'] == i]['Classic XPath'].values[0])
#                 print(col_a)
    
                try:
                    location = [a for a in flat_json_1.keys() if a.startswith(str(col_a))]
                    if len(location) == 1:
                        mapped_data.update({item : flat_json_1[col_a]})
                    else:
        #             print(location)
                        location_data = pd.Series({v: flat_json_1[v] for v in location})
            #             print(location_data)
                        location_df = pd.DataFrame([range(1, len(location_data.index)+1), location_data, location_data.index]).T

                        location_df['keyid'] = [re.findall(r'\d+', string)[0] for string in location_df[2]]
                        location_df['subfield'] = [string.split('].')[1] for string in location_df[2]]
                        loc = {}
            #             print(location_df)
                        for num, (g, h) in enumerate(location_df.groupby(by='keyid')):
                            loc.update({num: {key: val for key, val in zip(h['subfield'], h[1])}})
        #                     print()
    #                     print(loc)
                        mapped_data.update({item : loc})
                except:
                    pass
                
            elif item == 'startDate':
                col_a = xpath(relevant_fields_df[relevant_fields_df['Piece Name'] == field_list[item]]['Classic XPath'].values[0])
                try:
                    sdate = [a for a in flat_json_1.keys() if a.startswith(str(col_a))][0]
                    mapped_data.update({item : flat_json_1[sdate]})
#                     print(sdate)
                except:
                    pass
#                 
            elif item == 'endDate':
                col_a = xpath(relevant_fields_df[relevant_fields_df['Piece Name'] == field_list[item]]['Classic XPath'].values[0])
                try:
                    edate = [a for a in flat_json_1.keys() if a.startswith(str(col_a))][0]
                    mapped_data.update({item : flat_json_1[edate]})
#                     print(edate)
                except:
                    pass
    
            else:
#                 print(i)
                try:
                    col_a = xpath(relevant_fields_df[relevant_fields_df['Piece Name'] == field_list[item]]['Classic XPath'].values[0])
#                     print(col_a, '***')
#                     print()
                    mapped_data.update({item : flat_json_1[col_a]})
                except:
                    pass
            
    super_dict.update({n : mapped_data})

<ipython-input-593-e9a49ff8e2e2>:37: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  location_data = pd.Series({v: flat_json_1[v] for v in location})


In [594]:
# the data is mapped and dumped into json
with open(os.path.join(os.getcwd(), 'mapped_clinicaldata.json'), "w") as f:
    json.dump(super_dict, f, indent=4)

In [607]:
import json
import pandas as pd
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
import logging


# Preprocess relevant_fields_df into a dictionary for quick lookups
relevant_fields_dict = relevant_fields_df.set_index("Piece Name")["Classic XPath"].to_dict()

# Configure logging
logging.basicConfig(level=logging.ERROR)

## Manually written dict based on the keywords found from searching keys from example schema, required as the field names are different.
field_list = {'trialId':'nctId', 'title':'officialTitle', 'startDate':'startDateStruct', 'endDate':'completionDateStruct', 'phase':'Phase', 'principalInvestigator':['investigatorFullName',  'investigatorAffiliation'], 'locations':'locations\xa0⤷', 'eligibilityCriteria':'eligibilityCriteria'}

def flatten_json(y, parent_key='', sep='.'):
    """
    Flattens a nested JSON object into a single level.
    Keys will be in the form 'parent.child.grandchild'.
    """
    items = []
    for k, v in y.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                items.extend(flatten_json({f"{k}[{i}]": item}, parent_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


def xpath(a: str) -> str:
    """Modify XPath strings."""
    if '/' not in a:
        return a
    b = a.replace('/', '.')
    return b.replace('.Study.', '') if 'Study' in b else b

def process_list_field(i, flat_json_1):
    """Handle list-type fields like 'principalInvestigator'."""
    investigater = {}
    for f in i:
        col_a = xpath(relevant_fields_dict.get(f, ""))
        if col_a in flat_json_1:
            investigater[f] = flat_json_1[col_a]
    return investigater

def process_location_field(i, flat_json_1):
    """Handle 'locations' field."""
    col_a = xpath(relevant_fields_dict.get(i, ""))
    location = [key for key in flat_json_1.keys() if key.startswith(col_a)]
    if not location:
        return None
    if len(location) == 1:
        return flat_json_1[location[0]]

    location_data = pd.Series({v: flat_json_1[v] for v in location})
    location_df = pd.DataFrame(
        {
            "index": range(1, len(location_data.index) + 1),
            "value": location_data.values,
            "full_key": location_data.index,
        }
    )
    location_df["keyid"] = location_df["full_key"].str.extract(r"(\d+)")
    location_df["subfield"] = location_df["full_key"].str.split("].").str[1]
    loc = {}
    for num, (keyid, group) in enumerate(location_df.groupby("keyid")):
        loc[num] = dict(zip(group["subfield"], group["value"]))
    return loc


def process_mapping(combined_study):
    # Process studies sequentially
    super_dict = {}
    for n, study in enumerate(combined_study):
        print(n)
        flat_json_1 = flatten_json(study)
        mapped_data = {}
        for item in field_list:
            print(item)
            i = field_list[item]
            try:
                if isinstance(i, list):
                    mapped_data[item] = process_list_field(i, flat_json_1)
                elif item == "locations":
                    mapped_data[item] = process_location_field(i, flat_json_1)
                elif item in ["startDate", "endDate"]:
                    col_a = xpath(relevant_fields_dict.get(field_list[item], ""))
                    sdate = next((key for key in flat_json_1.keys() if key.startswith(col_a)), None)
                    if sdate:
                        mapped_data[item] = flat_json_1[sdate]
                else:
                    col_a = xpath(relevant_fields_dict.get(field_list[item], ""))
                    if col_a in flat_json_1:
                        mapped_data[item] = flat_json_1[col_a]
            except Exception as e:
                logging.error(f"Error processing {item} for study {n}: {e}")
        super_dict[n] = mapped_data
        return super_dict

In [608]:
super_dict2 = process_mapping(combined_studies)

0
trialId
title
startDate
endDate
phase
principalInvestigator
locations
eligibilityCriteria


In [604]:
len(super_dict2)

1

In [ ]:
# #This was a test to filter out classic XPaths, as some paths had "/", dont require now

# relevant_fields_df = pd.concat(relevant_fields.values())

# print(relevant_fields_df)
# # Keep rows where column 2 does NOT contain '/'
# filtered_df = relevant_fields_df[~relevant_fields_df["Classic XPath"].str.contains("/", regex=False)]
# filtered_df.index = range(1, len(filtered_df.index)+1)
# print(filtered_df)

# filtered_schema_v1 = {}
# for n, x in enumerate(combined_studies):
#     flat_json = flatten_json(combined_studies[n])
# #     print(flat_json)
# #     print(n)
#     results = {}
#     for keys in filtered_df['Classic XPath']:
# #         print(keys)
        
        
#         matches = {k: v for k, v in flat_json.items() if k.startswith(keys)}
# #         print(matches)
#         if matches:
# #             print(f"Matches for {keys}: {matches}")
#             results.update({keys: matches})  # Store matches
# #         else:
# #             print(f"No matches for {keys}")
# #     pp.pprint(results)
#     filtered_schema_v1.update({n: results})
#     break

# filtered_schema_v2 = {}
# for n, (x, y) in enumerate(filtered_schema_v1.items()):
    
#     nct_id = y['protocolSection.identificationModule.nctId']
    
#     print(nct_id)
    
#     # process locations data
#     location_data = pd.Series(y['protocolSection.contactsLocationsModule.locations'])
#     location_df = pd.DataFrame([range(1, len(location_data.index)+1), location_data, location_data.index]).T
    
#     location_df['keyid']z = [re.findall(r'\d+', string)[0] for string in location_df[2]]
#     location_df['subfield'] = [string.split('].')[1] for string in location_df[2]]

# #     print(location_df)
#     for g, h in location_df.groupby(by='keyid'):
#         print({key: val for key, val in zip(h['subfield'], h[1])})
#     break

In [346]:
from functools import reduce

In [247]:
field_list = {'trialId':'nctId', 'title':'officialTitle', 'startDate':'StartDate', 'endDate':'CompletionDate', 'phase':'Phase', 'principalInvestigator':['investigatorFullName',  'investigatorAffiliation'], 'locations':'locations\xa0⤷', 'eligibilityCriteria':'eligibilityCriteria'}

for i, j in field_list.items():
    
#     print(i)
    
    if isinstance(j, list):
#         print(len(j))
#         print('yes')

        for k in j:
#             print(j, k)
            field_name = relevant_fields[i]
            field_xpath = field_name[field_name['Piece Name'] == k]['Classic XPath'].values
            print(field_xpath)
        
#             dict_fields = [list(filter(None, re.split(r"[./]", x))) for x in field_xpath]
# #             print(dict_fields)
            
#             for key in dict_fields:
#                 print(key)
#                 try:
#                     found_terms = reduce(lambda d, k: d[k], key, combined_studies[0])
#                     print(found_terms)
#                 except:
#                     pass
            
#             if len(dict_fields) > 1:
#                 print('found', dict_fields)
                
                
                
#             print([x.split('.') for x in field_xpath])

        
    else:
        print(i)
        field_name = relevant_fields[i]
        field_xpath = field_name[field_name['Piece Name'] == j]['Classic XPath'].values
        print(field_xpath)
#         dict_fields = [list(filter(None, re.split(r"[./]", x))) for x in field_xpath]
#         print(dict_fields)
#         for key in dict_fields:
#             print(key)
#             try:
#                 found_terms = reduce(lambda d, k: d[k], key, combined_studies[0])
#                 print(found_terms)
#             except:
#                 pass

    
    print()

trialId
['protocolSection.identificationModule.nctId'
 'protocolSection.statusModule.expandedAccessInfo.nctId']

title
['protocolSection.identificationModule.officialTitle']

startDate
['/Study/ProtocolSection/StatusModule/StartDateStruct/StartDate']

endDate
['/Study/ProtocolSection/StatusModule/CompletionDateStruct/CompletionDate']

phase
['/Study/ProtocolSection/DesignModule/PhaseList/Phase']

['protocolSection.sponsorCollaboratorsModule.responsibleParty.investigatorFullName']
['protocolSection.sponsorCollaboratorsModule.responsibleParty.investigatorAffiliation']

locations
['protocolSection.contactsLocationsModule.locations']

eligibilityCriteria
['protocolSection.eligibilityModule.eligibilityCriteria']



In [236]:
list_dfs = [pd.DataFrame(combined_studies[n]['protocolSection']) for n, x in enumerate(combined_studies)]
list_dfs
concat_dfs = pd.concat(list_dfs, axis=0)

In [240]:
concat_dfs['identificationModule']['nctId']

nctId    NCT01524276
nctId    NCT04964960
nctId    NCT06649682
nctId    NCT04876482
nctId    NCT06636682
            ...     
nctId    NCT05819645
nctId    NCT06650345
nctId    NCT03681561
nctId    NCT06650553
nctId    NCT03888651
Name: identificationModule, Length: 572, dtype: object

In [174]:
relevant_fields['title']

,Piece Name,Classic XPath
30,briefTitle,protocolSection.identificationModule.briefTitle
31,BriefTitle,/Study/ProtocolSection/IdentificationModule/Br...
32,officialTitle,protocolSection.identificationModule.officialT...
33,OfficialTitle,/Study/ProtocolSection/IdentificationModule/Of...
144,investigatorTitle,protocolSection.sponsorCollaboratorsModule.res...
145,ResponsiblePartyInvestigatorTitle,/Study/ProtocolSection/SponsorCollaboratorsMod...
148,oldNameTitle,protocolSection.sponsorCollaboratorsModule.res...
149,ResponsiblePartyOldNameTitle,/Study/ProtocolSection/SponsorCollaboratorsMod...


In [95]:
datastruct_protocol = pd.read_csv(os.getcwd()+'\\datastruct\\Protocolsection.csv', header=0, skiprows=1)
# print(datastruct_protocol)



['eligibilityCriteria', 'EligibilityCriteria']


In [69]:
for n, x in enumerate(combined_studies[:3]):
    print(x.keys(), '*************')
    root_dict = combined_studies[n]
    protocol_section =  root_dict[list(x.keys())[0]]
    for a, b in protocol_section.items():
        print(protocol_section[a].keys())
    print()
#     break

dict_keys(['protocolSection', 'derivedSection', 'hasResults']) *************
dict_keys(['nctId', 'orgStudyIdInfo', 'organization', 'briefTitle', 'officialTitle', 'acronym'])
dict_keys(['statusVerifiedDate', 'overallStatus', 'expandedAccessInfo', 'startDateStruct', 'primaryCompletionDateStruct', 'completionDateStruct', 'studyFirstSubmitDate', 'studyFirstSubmitQcDate', 'studyFirstPostDateStruct', 'lastUpdateSubmitDate', 'lastUpdatePostDateStruct'])
dict_keys(['responsibleParty', 'leadSponsor'])
dict_keys(['oversightHasDmc', 'isUsExport'])
dict_keys(['briefSummary'])
dict_keys(['conditions'])
dict_keys(['studyType', 'designInfo', 'enrollmentInfo'])
dict_keys(['primaryOutcomes'])
dict_keys(['eligibilityCriteria', 'healthyVolunteers', 'sex', 'stdAges', 'studyPopulation', 'samplingMethod'])
dict_keys(['centralContacts', 'locations'])
dict_keys(['references'])
dict_keys(['ipdSharing'])

dict_keys(['protocolSection', 'resultsSection', 'documentSection', 'derivedSection', 'hasResults']) *******

In [139]:
combined_studies[0]['protocolSection']['identificationModule']['nctId']

'NCT01524276'

In [173]:
from functools import reduce

for key in [x.split('.') for x in nct_id][:1]:
#     print(key)
    print(reduce(lambda d, k: d[k], key, combined_studies[0]))
    

NCT01524276


In [255]:
def flatten_json(y, parent_key='', sep='.'):
    """
    Flattens a nested JSON object into a single level.
    Keys will be in the form 'parent.child.grandchild'.
    """
    items = []
    for k, v in y.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                items.extend(flatten_json({f"{k}[{i}]": item}, parent_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


In [245]:
# flat_json = flatten_json(combined_studies[0])
# print(flat_json)